## Introduktion

Denne lektion vil dække:
- Hvad funktionskald er, og hvornår de bruges
- Hvordan man opretter et funktionskald med OpenAI
- Hvordan man integrerer et funktionskald i en applikation

## Læringsmål

Når du har gennemført denne lektion, vil du vide hvordan og forstå:

- Formålet med at bruge funktionskald
- Opsætning af Funktionskald med OpenAI-tjenesten
- Designe effektive funktionskald til din applikations brugsscenarie


## Forståelse af funktionskald

I denne lektion vil vi bygge en funktion til vores uddannelses-startup, der gør det muligt for brugere at bruge en chatbot til at finde tekniske kurser. Vi vil anbefale kurser, der passer til deres færdighedsniveau, nuværende rolle og interesseområde inden for teknologi.

For at gennemføre dette vil vi bruge en kombination af:
 - `OpenAI` til at skabe en chatoplevelse for brugeren
 - `Microsoft Learn Catalog API` til at hjælpe brugere med at finde kurser baseret på deres forespørgsel
 - `Function Calling` til at tage brugerens forespørgsel og sende den til en funktion, der laver API-forespørgslen.

Lad os starte med at se på, hvorfor vi overhovedet vil bruge funktionskald:

print("Beskeder i næste forespørgsel:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # få et nyt svar fra GPT, hvor den kan se funktionssvaret


print(second_response.choices[0].message)


### Hvorfor Function Calling

Hvis du har gennemført en anden lektion i dette kursus, forstår du sikkert allerede, hvor kraftfulde Large Language Models (LLMs) er. Forhåbentlig kan du også se nogle af deres begrænsninger.

Function Calling er en funktion i OpenAI Service, der er udviklet for at løse følgende udfordringer:

Uensartet svarformat:
- Før function calling var svarene fra en large language model ustrukturerede og uensartede. Udviklere var nødt til at skrive kompleks valideringskode for at håndtere alle variationer i outputtet.

Begrænset integration med eksterne data:
- Før denne funktion var det svært at inddrage data fra andre dele af en applikation i en chatkontekst.

Ved at standardisere svarformater og muliggøre problemfri integration med eksterne data, gør function calling udviklingen nemmere og mindsker behovet for ekstra valideringslogik.

Brugere kunne ikke få svar som "Hvad er vejret lige nu i Stockholm?". Det skyldes, at modellerne kun havde adgang til data fra det tidspunkt, de blev trænet på.

Lad os se på eksemplet nedenfor, som illustrerer dette problem:

Lad os sige, at vi vil oprette en database med elevdata, så vi kan foreslå det rigtige kursus til dem. Nedenfor har vi to beskrivelser af elever, der minder meget om hinanden i forhold til de data, de indeholder.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Vi vil gerne sende dette til en LLM for at analysere dataene. Dette kan senere bruges i vores applikation til at sende det til et API eller gemme det i en database.

Lad os lave to identiske prompts, hvor vi instruerer LLM’en om, hvilke oplysninger vi er interesserede i:


Vi ønsker at sende dette til en LLM for at analysere de dele, der er vigtige for vores produkt. Så vi kan oprette to identiske prompts for at instruere LLM'en:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Efter at have oprettet disse to prompts, sender vi dem til LLM ved at bruge `openai.ChatCompletion`. Vi gemmer prompten i variablen `messages` og tildeler rollen til `user`. Dette er for at efterligne en besked fra en bruger, der skrives til en chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Selvom promptene er de samme og beskrivelserne ligner hinanden, kan vi få forskellige formater af `Grades`-egenskaben.

Hvis du kører ovenstående celle flere gange, kan formatet være `3.7` eller `3.7 GPA`.

Det skyldes, at LLM'en tager ustrukturerede data i form af den skrevne prompt og også returnerer ustrukturerede data. Vi har brug for et struktureret format, så vi ved, hvad vi kan forvente, når vi gemmer eller bruger disse data.

Ved at bruge funktionel kald kan vi sikre, at vi får strukturerede data tilbage. Når vi bruger funktionel kald, kalder eller kører LLM'en faktisk ikke nogen funktioner. I stedet opretter vi en struktur, som LLM'en skal følge i sine svar. Vi bruger derefter de strukturerede svar til at vide, hvilken funktion vi skal køre i vores applikationer.


![Funktionskaldsflowdiagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.da.png)


### Brugsscenarier for brug af funktionskald

**Kald til eksterne værktøjer**  
Chatbots er gode til at besvare brugernes spørgsmål. Ved at bruge funktionskald kan chatbots bruge beskeder fra brugerne til at udføre bestemte opgaver. For eksempel kan en studerende bede chatbotten om at "Sende en mail til min underviser og sige, at jeg har brug for mere hjælp til dette emne". Dette kan udløse et funktionskald til `send_email(to: string, body: string)`

**Oprette API- eller databaseforespørgsler**  
Brugere kan finde information ved at bruge naturligt sprog, som bliver omdannet til en formateret forespørgsel eller API-anmodning. Et eksempel kunne være en lærer, der spørger "Hvem er de elever, der har afleveret den sidste opgave", hvilket kan kalde en funktion ved navn `get_completed(student_name: string, assignment: int, current_status: string)`

**Oprettelse af strukturerede data**  
Brugere kan tage et tekstafsnit eller en CSV-fil og bruge LLM til at udtrække vigtige oplysninger fra det. For eksempel kan en studerende omdanne en Wikipedia-artikel om fredsaftaler til at lave AI-flashkort. Dette kan gøres ved at bruge en funktion kaldet `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Oprettelse af dit første funktionskald

Processen med at oprette et funktionskald består af 3 hovedtrin:
1. Kald Chat Completions API'et med en liste over dine funktioner og en brugermeddelelse
2. Læs modellens svar for at udføre en handling, dvs. køre en funktion eller et API-kald
3. Lav endnu et kald til Chat Completions API'et med svaret fra din funktion for at bruge den information til at skabe et svar til brugeren.


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.da.png)


### Elementer i et funktionskald

#### Brugerinput

Det første skridt er at oprette en brugermeddelelse. Dette kan gøres dynamisk ved at tage værdien fra et tekstinput, eller du kan tildele en værdi her. Hvis det er første gang, du arbejder med Chat Completions API'en, skal vi definere `role` og `content` for beskeden.

`role` kan enten være `system` (opretter regler), `assistant` (modellen) eller `user` (slutbrugeren). Til funktionskald sætter vi dette som `user` og tilføjer et eksempelspørgsmål.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Oprettelse af funktioner.

Nu skal vi definere en funktion og dens parametre. Vi bruger kun én funktion her, som hedder `search_courses`, men du kan sagtens oprette flere funktioner.

**Vigtigt**: Funktioner bliver inkluderet i systembeskeden til LLM’en og vil tælle med i det antal tokens, du har til rådighed.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitioner**

Strukturen for funktionsdefinitioner har flere niveauer, hver med sine egne egenskaber. Her er en oversigt over den indlejrede struktur:

**Egenskaber for funktion på øverste niveau:**

`name` - Navnet på den funktion, vi ønsker at få kaldt.

`description` - Dette er beskrivelsen af, hvordan funktionen fungerer. Her er det vigtigt at være specifik og tydelig.

`parameters` - En liste over værdier og format, som du ønsker, at modellen skal give i sit svar.

**Egenskaber for parameters-objektet:**

`type` - Datatypen for parameters-objektet (typisk "object")

`properties` - Liste over de specifikke værdier, som modellen vil bruge i sit svar

**Egenskaber for individuelle parametre:**

`name` - Defineres implicit af egenskabens nøgle (f.eks. "role", "product", "level")

`type` - Datatypen for denne specifikke parameter (f.eks. "string", "number", "boolean")

`description` - Beskrivelse af den specifikke parameter

**Valgfrie egenskaber:**

`required` - Et array, der angiver, hvilke parametre der er påkrævet for at funktionen kan udføres


### Sådan foretager du funktionskald
Efter at have defineret en funktion, skal vi nu inkludere den i kaldet til Chat Completion API'et. Det gør vi ved at tilføje `functions` til forespørgslen. I dette tilfælde `functions=functions`.

Der er også mulighed for at sætte `function_call` til `auto`. Det betyder, at vi lader LLM'en vælge, hvilken funktion der skal kaldes, baseret på brugerens besked, i stedet for at vi selv vælger det.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nu lad os se på svaret og se, hvordan det er formateret:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du kan se, at navnet på funktionen bliver kaldt, og ud fra brugerens besked var LLM i stand til at finde dataene til at udfylde argumenterne til funktionen.


## 3. Integrering af funktionskald i en applikation.

Når vi har testet det formaterede svar fra LLM'en, kan vi nu integrere det i en applikation.

### Håndtering af flowet

For at integrere dette i vores applikation, lad os tage følgende skridt:

Først laver vi kaldet til OpenAI-tjenesterne og gemmer beskeden i en variabel kaldet `response_message`.


In [ ]:
response_message = response.choices[0].message

Nu vil vi definere funktionen, der vil kalde Microsoft Learn API'et for at hente en liste over kurser:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Som en god praksis vil vi derefter se, om modellen ønsker at kalde en funktion. Derefter opretter vi en af de tilgængelige funktioner og matcher den med den funktion, der bliver kaldt.
Vi tager så argumenterne fra funktionen og matcher dem med argumenterne fra LLM.

Til sidst tilføjer vi beskeden om funktionskaldet og de værdier, der blev returneret af `search_courses`-beskeden. Dette giver LLM alle de oplysninger, den har brug for
til at svare brugeren med naturligt sprog.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodeudfordring

Godt arbejde! For at fortsætte din læring om OpenAI Function Calling kan du bygge: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Flere parametre til funktionen, som kan hjælpe brugere med at finde flere kurser. Du kan finde de tilgængelige API-parametre her:
 - Opret et andet funktionskald, der tager mere information fra brugeren, såsom deres modersmål
 - Tilføj fejlhåndtering, hvis funktionskaldet og/eller API-kaldet ikke returnerer nogen relevante kurser



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiske oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel, menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå ved brug af denne oversættelse.
